## Aprendizaje de máquina II
### Carrera de especialización en inteligencia artificial  

#### **VERSIONADO DE MODELOS USANDO MLFLOW**

Este ejemplo pertenece a la [documentación](https://github.com/mlflow/mlflow/tree/master/examples/sklearn_elasticnet_wine) de MLflow, con algunas modificaciones para trackear nuestros modelos utilizando sqlite.  

Para poder reproducir los resultados vistos en clase seguir los siguientes pasos:

* Si bien no es obligatorio, es **altamente** recomendable crearse un nuevo ambiente para administrar las dependencias y asegurar la correcta ejecución.  
Si estamos utilizando conda podemos crear un nuevo entorno con el comando:
`conda create -n mlflow python=3.8`  
Al ejecutar ese comando se nos creará un ambiente llamado "mlflow" (cambiar el nombre si se lo desea) con python versión 3.8.  

Podemos activar nuestro nuevo ambiente ejecutando:  
`conda activate mlflow`  
_(En caso de haber elegido otro nombre para el ambiente, reemplazar "mlflow" por el nombre que hayamos elegido)_


Para completar la instalación del ambiente debemos instalar las siguientes dependencias:  

  - scikit-learn==1.2.0
  - mlflow
  - pandas

Para esto podemos utilizar `pip install nombre_de_la_biblioteca` desde la consola de conda.  

* Luego de configurar nuestro ambiente debemos abrir la command prompt de conda y movernos hacia el directorio en donde tengamos guardado este notebook. Como recomendación, guardarlo en una carpeta exclusiva, ya que se nos irán generando algunos archivos complementarios para poder realizar el tutorial.

En caso de que no hayan navegado por una consola de comandos, [acá](http://www.falconmasters.com/offtopic/como-utilizar-consola-de-windows/#:~:text=Para%20acceder%20a%20ella%20lo,en%20la%20consola%20de%20windows.) hay un breve tutorial con los comandos más útiles.


* Una vez dentro de la carpeta donde almacenamos este notebook, debemos indicarle a mlflow que vamos a utilizar SQLite como backend para almacenar nuestros modelos registrados. Para ello, desde la command prompt de conda debemos ejecutar:  
`mlflow server --backend-store-uri sqlite:///mydb.sqlite`  
Luego de ejecutar ese comando veremos que en la carpeta se crearán una carpeta donde se almacenarám los artefactos de mlflow y una base de datos para el model registry.  
También debemos ver en la consola el siguiente mensaje:
`INFO:waitress:Serving on http://127.0.0.1:5000`

* Esa dirección IP corresponde a nuestro localhost y el número 5000 al número de puerto donde podremos consultar la UI de mlflow.  
Si copiamos y pegamos esa dirección http en algún buscador web, podremos acceder a la UI.  



In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

In [ ]:
# !mlflow server --backend-store-uri sqlite:///mydb.sqlite

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("Wine_prediction_experiment")

KeyboardInterrupt: 

In [1]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha = 0.2
    l1_ratio = 0.1

    # # Otra opcion
    # mlflow.start_run()
    # # Codigo
    # mlflow.end_run()
    
    # Pongo nombre a la ejecucion
    # with mlflow.start_run(run_name = "lr_model_run"):
    
    # Usa un nombre aleatorio de dos palabras con 3 numeros
    with mlflow.start_run():        
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        IS_AUTOREGISTER = False
        # Model registry does not work with file store
        if (tracking_url_type_store != "file") and IS_AUTOREGISTER:
            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
        else:
            mlflow.sklearn.log_model(lr, "model")
        

NameError: name 'logging' is not defined

### Agrego ajuste de HPs

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Crear un diccionario con los parámetros que deseamos optimizar
param_dist = {
    "alpha": uniform(0, 1),
    "l1_ratio": uniform(0, 1),
    "fit_intercept": [True, False],
    "max_iter": [1000, 5000, 10000],
}

In [12]:
mlflow.sklearn.autolog()
tags = {
    "Country": "Brasil",
    "Model Type": "ElasticNet",
}
with mlflow.start_run():
    mlflow.set_tags(tags)
    eNet_model = ElasticNet(random_state=42)

    randomized_search = RandomizedSearchCV(
        estimator=eNet_model,
        param_distributions=param_dist,
        n_iter=10,
        cv=5,
        n_jobs=-1
    )

    randomized_search.fit(train_x, train_y)

    predicted_qualities = randomized_search.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # No hacen falta si hay mlflow.sklearn.autolog(). Ya se loguean automaticamente
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    IS_AUTOREGISTER = False
    # Model registry does not work with file store
    if (tracking_url_type_store != "file") and IS_AUTOREGISTER:
        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        mlflow.sklearn.log_model(randomized_search, "model", registered_model_name="ElasticnetWineModel")
    else:
        mlflow.sklearn.log_model(randomized_search, "model")

AssertionError: C:\Users\EZEQUIEL\AppData\Local\Programs\Python\Python310\lib\distutils\core.py

### ¿Cómo uso los modelos registrados para generar predicciónes?

In [9]:
# Cargo modelos según su estado
model_production = mlflow.pyfunc.load_model('models:/wine_predictor/production')
model_staging = mlflow.pyfunc.load_model('models:/wine_predictor/staging')

# Cargo modelos según su número de versión
model_v2 = mlflow.pyfunc.load_model('models:/wine_predictor/2')

2023/05/13 12:25:02 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2023/05/13 12:25:03 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2023/05/13 12:25:03 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(mode

In [7]:
# Cargo modelo según el Run ID en que fué creado
logged_model = 'runs:/db0a15fe573f413798bf4b5d786d81d8/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

2023/05/13 12:23:53 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Pregunta: ¿Cuál de las formas antes vistas es preferible?

In [12]:
model_v2.predict(test_x)

array([5.87293694, 4.96801408, 5.82508844, 6.40678195, 5.33057884,
       5.41679834, 5.92877456, 6.05740966, 5.38539593, 5.5090109 ,
       5.95979193, 5.60860378, 6.01295389, 6.38858278, 5.30502856,
       5.9144681 , 5.93526841, 5.11970663, 5.31824978, 5.26800046,
       5.28496281, 5.47898316, 4.98048242, 5.47295505, 6.20779065,
       5.51503779, 5.57965964, 5.32791035, 5.87047863, 5.41816862,
       5.53552732, 5.68004797, 5.25329461, 5.13154251, 5.17504122,
       5.54411597, 6.56699112, 5.17795214, 5.8256639 , 5.54709817,
       5.38383908, 5.67250473, 5.45655693, 5.74704323, 5.93685271,
       5.18089733, 5.65355992, 6.00904975, 6.5673674 , 5.09459259,
       6.05740966, 5.28553516, 5.30885492, 6.23891235, 5.90650865,
       5.11997504, 5.68045932, 5.69473777, 6.00108828, 5.49909503,
       6.01979187, 5.16875806, 5.69799429, 5.19345097, 5.77796839,
       5.64839021, 5.17301518, 5.57675441, 6.028012  , 6.32678032,
       5.34506746, 5.18357159, 5.58654886, 5.95740748, 5.86406

#### **ACTIVIDAD:** levantar el host de mlflow, entrenar un RandomForest, registrarlo, pasarlo a producción y realizar inferencia con ese modelo.